<a href="https://colab.research.google.com/github/imortalfunction/kaggle-study/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

필요 라이브러리 가져오기

In [77]:

## 데이터 분석 관련
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

## 데이터 시각화 관련
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid') # matplotlib의 스타일에 관련한 함
## 그래프 출력에 필요한 IPython 명령어
%matplotlib inline

## Scikit-Learn의 다양한 머신러닝 모듈을 불러옵니다.
## 분류 알고리즘 중에서 선형회귀, 서포트벡터머신, 랜덤포레스트, K-최근접이웃 알고리즘을 사용해보려고 합니다.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

구글 마운트

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


데이터 읽기, csv를 보편적으로 많이 사용한다.

In [78]:

# 데이터를 가져오기.
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test.csv")

# 데이터 미리보기
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


info 메서드로 데이터 확인

In [79]:
train_df.info()
print('-'*20)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    

필요 없는 부분 드롭

In [80]:
train_df = train_df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
test_df = test_df.drop(['Name','Ticket'], axis=1)

Pclass :
1등석, 2등석, 3등석 정보
각 클래스 별 갯수 출력

In [81]:
train_df['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

Pclass를 범주형 데이터로 취급하여 인코딩(pd.get_dumies())사용

In [82]:
pclass_train_dummies = pd.get_dummies(train_df['Pclass'])
pclass_test_dummies = pd.get_dummies(test_df['Pclass'])

train_df.drop(['Pclass'], axis=1, inplace=True)
test_df.drop(['Pclass'], axis=1, inplace=True)

train_df = train_df.join(pclass_train_dummies)
test_df = test_df.join(pclass_test_dummies)

Sex : 탑승자의 성별로서 Pclass와 마찬가지로 범주형 데이터로 인코딩

In [83]:
sex_train_dummies = pd.get_dummies(train_df['Sex'])
sex_test_dummies = pd.get_dummies(test_df['Sex'])

sex_train_dummies.columns = ['Female', 'Male']
sex_test_dummies.columns = ['Female', 'Male']

train_df.drop(['Sex'], axis=1, inplace=True)
test_df.drop(['Sex'], axis=1, inplace=True)

train_df = train_df.join(sex_train_dummies)
test_df = test_df.join(sex_test_dummies)

Age : 탑승자의 나이, 연속형 데이터로서 데이터 처리 이전에 NaN 데이터를 평균값으로 채워 넣음

In [84]:
train_df["Age"].fillna(train_df["Age"].mean() , inplace=True)
test_df["Age"].fillna(train_df["Age"].mean() , inplace=True)

Fare : 탑승료, 빈 데이터는 무단 탑승으로 처리해 0값을 할당

In [85]:
test_df["Fare"].fillna(0, inplace=True)

cabin : 객실, 데이터 셋에 NaN 값이 많기에 가감히 소거

In [86]:
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

Embarked : 탑승 항구, 항구는 S, C, Q 세종류가 있으며 데이터가 빈 곳이 있는데 이를 제일 많은 S로 채워 넣는다.

In [87]:
train_df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [88]:
test_df['Embarked'].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [89]:
train_df["Embarked"].fillna('S', inplace=True)
test_df["Embarked"].fillna('S', inplace=True)

In [90]:
embarked_train_dummies = pd.get_dummies(train_df['Embarked'])
embarked_test_dummies = pd.get_dummies(test_df['Embarked'])

embarked_train_dummies.columns = ['S', 'C', 'Q']
embarked_test_dummies.columns = ['S', 'C', 'Q']

train_df.drop(['Embarked'], axis=1, inplace=True)
test_df.drop(['Embarked'], axis=1, inplace=True)

train_df = train_df.join(embarked_train_dummies)
test_df = test_df.join(embarked_test_dummies)

데이터 나누기

In [91]:
X_train = train_df.drop("Survived",axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId",axis=1).copy()

로지스틱 회귀 : 분류 문제에 사용되는 대표적인 알고리즘으로 이진 분류에 특화되어 있다. 입력 변수의 선형 결합을 시그모이드 함수에 적용하여 확률 값을 추정하는 방식이다. 시그모이드 함수는 0과 1사이의 값을 출력하는 함수이다.

In [92]:
X_train.columns = X_train.columns.astype(str)

X_test.columns = X_test.columns.astype(str)


logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)

logreg.score(X_train, Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8047138047138047

서포트 백터 머신(SVM) 사용 : 분류와 회귀 분석에 사용되는 감독 학습 모델로서 두 카테고리로 나누는 결정 경계(하이퍼플레인)를 찾는 알고리즘이다.

주요 구성 요소

1. support vectors : 결정 경계를 구성하는데 필요한 데이터 포인트, 결정 경계와 가장 가까운 포인트들로, 결정 경계의 위치와 방향을 결정

2. Hyperplane :  SVM 분류를 위해 사용하는 결정 경계. n차원 공간에서 n-1 차원의 평면으로 두 카테고리로 나눈다.

3. Margin :  결정 경계와 서포트 벡터 사이의 거리로 SVM의 목표는 이 마진을 최대화 시키는 것이다.



In [93]:
svc = SVC()

svc.fit(X_train, Y_train)

Y_pred = svc.predict(X_test)

svc.score(X_train, Y_train)

0.6868686868686869

랜덤 포레스트 : 여러개의 결정 트리를 학습하여, 각 트리의 예측을 종합해 최종 예측을 수행하는 알고리즘이다.

작동 방식
1. 부트스트랩 샘플 생성 : 훈련 데이터셋에서 무작위로 데이터를 선택(중복 허용)해 새로운 데이터셋을 생성한다. 각 결정 트리가 서로 다른 데이터셋으로 학습하게 한다.
2. 각 트리의 학습 : 생성된 각각의 부트스트랩 샘플에 대해 결정 트리를 학습시킨다. 각 노드에서 모든 특징을 고려하는 대신, 무작위로 선택된 소수의 특징만을 고려한다. 이는 각 트리가 데이터에 대해 과적합되는 ㄱ서을 방지하고, 다양한 특징을 고려해 각 트리가 서로 다른 특징에 대해 학습하게 하는 것이다.
3. 예측 : 새로운 데이터 포인트에 대한 예측을 수행할 때, 각 트린느 독립적으로 예측을 수행하고 그 예측을 투표하여 최종 예측 결과를 결정한다. 분류 문제일시 가장 많은 표를 얻은 클래스가 최종 예측 클래스가 된다. 회귀 문제에선 각 트리의 예측값의 평균이 최종 예측값이 된다.


In [94]:
random_forest = RandomForestClassifier(n_estimators=100)

random_forest.fit(X_train, Y_train)

Y_pred = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)

0.9820426487093153

K-NN(Nearest Neighbors) : 분류, 회귀 모두 사용 가능한 알고리즘으로 거리를 기준으로 가장 가까운 K개의 데이터 포인트들을 통해 클래스를 에측하는 알고리즘이다.

In [95]:
knn = KNeighborsClassifier(n_neighbors = 3)

knn.fit(X_train, Y_train)

Y_pred = knn.predict(X_test)

knn.score(X_train, Y_train)

0.835016835016835

4가지의 알고리즘을 비교했을때 가장 우수한 결과를 내는 랜덤 포레스트를 선정하여 파일을 저장한다.

In [96]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)

submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('titanic.csv', index=False)

출처 : https://www.kaggle.com/code/subinium/subinium-tutorial-titanic-beginner